In [1]:
import dask_ml
dask_ml.__file__

'/Users/scott/Developer/stsievert/dask-ml/dask_ml/__init__.py'

In [2]:
from distributed import Client, LocalCluster
# cluster = LocalCluster(n_workers=25)
# client = Client(cluster)
client = Client("localhost:8786")
client

/Users/scott/anaconda3/lib/python3.6/site-packages/distributed/utils.py:136: RuntimeWarning: Couldn't detect a suitable IP address for reaching '8.8.8.8', defaulting to '127.0.0.1': [Errno 51] Network is unreachable
  RuntimeWarning,


Client Scheduler: tcp://localhost:8786 Dashboard: http://localhost:8787/status,Cluster Workers: 10 Cores: 10 Memory: 21.47 GB


In [9]:
import json
from sys import getsizeof

today = "2019-06-26"
random_state = 152
with open(f"out/{today}/-hyperband-{random_state}-history.json", "r") as f:
    history = json.load(f)
# _ = [h.pop("params", None) for h in history]
_bytes = getsizeof(history)
print(f"{_bytes / 2014:0.2f} kB")
history[0]

1.64 kB


{'model_id': 'bracket=1-0',
 'params': {'train_split': None,
  'optimizer__weight_decay': 0,
  'optimizer__nesterov': True,
  'optimizer__momentum': 0.16316316316316315,
  'optimizer__lr': 1.666179250391986,
  'optimizer': 'Adam',
  'module__init': 'xavier_uniform_',
  'module__activation': 'PReLU',
  'batch_size': 64},
 'partial_fit_calls': 1,
 'partial_fit_time': 17.463095903396606,
 'score': -0.16461865603923798,
 'score_time': 17.51579713821411,
 'elapsed_wall_time': 185.12997460365295,
 'bracket': 1}

In [4]:
from sklearn.base import BaseEstimator
import numpy as np
import toolz

def _same_params(p1, p2):
    p1keys = set(p1.keys())
    p2keys = set(p2.keys())
    assert p1keys == p2keys
    for k in p1.keys():
        if isinstance(p1[k], float) and not np.allclose(p1[k], p2[k]):
            return False
        elif p1[k] != p2[k]:
            return False
    return True
    
def _get_model_history(history, params):
    model_histories = toolz.groupby("model_id", history)
    same_params = {k: _same_params(v[0]["params"], params) for k, v in model_histories.items()}
    assert sum(same_params.values()) == 1
    model_id = [k for k, v in same_params.items() if v][0]
    return model_histories[model_id]
    
params = {
    'module_init': ['xavier_uniform_',
                     'xavier_normal_',
                     'kaiming_uniform_',
                     'kaiming_normal_',
                    ],
    'module_activation': ['ReLU', 'LeakyReLU', 'ELU', 'PReLU'],
    'optimizer': ["SGD"] * 5 + ["Adam"] * 2,
    'batch_size': [32, 64, 128, 256, 512],
    'optimizer_lr': np.logspace(1, -1.5, num=1000),
    'optimizer_weight_decay': [0]*200 + np.logspace(-5, -3, num=1000).tolist(),
    'optimizer_nesterov': [True],
    'optimizer_momentum': np.linspace(0, 1, num=1000),
    'train_split': [None],
}
   

In [5]:
from distributed.utils import sleep
 
class ReplayModel(BaseEstimator):
    def __init__(
        self,
        history,
        module_init=None,
        module_activation=None,
        optimizer=None,
        optimizer_lr=None,
        batch_size=None,
        optimizer_weight_decay=None,
        optimizer_nesterov=None,
        optimizer_momentum=None,
        train_split=None,
    ):
        self.history = history
        self._pf_calls = 0
        
        self.module_init = module_init
        self.module_activation = module_activation
        
        self.optimizer = optimizer
        self.optimizer_lr = optimizer_lr
        self.batch_size = batch_size
        
        self.optimizer_weight_decay = optimizer_weight_decay
        self.optimizer_nesterov = optimizer_nesterov
        self.optimizer_momentum = optimizer_momentum
        
        self.train_split = train_split
        
    def partial_fit(self, X, y):
        self._pf_calls += 1
        sleep(1)
        return self
    
    def fit(self, X, y):
        return self
    
    def _get_formatted_keys(self):
        params = self.get_params()
        params.pop("history")
        new_params = {}
        for k, v in params.items():
            if "module" in k:
                k = "module__" + "_".join(k.split("_")[1:])
            if "optimizer" in k and k != "optimizer":
                k = "optimizer__" + "_".join(k.split("_")[1:])
            new_params[k] = v
        return new_params
    
    def score(self, X, y):
        model_history = _get_model_history(self.history, self._get_formatted_keys())
        valid = [h for h in model_history if h["partial_fit_calls"] == self._pf_calls]
        return valid[0]["score"]

model = ReplayModel(history)

In [7]:
from dask_ml.model_selection import HyperbandSearchCV
import scipy.stats
from sklearn.datasets import make_classification

search = HyperbandSearchCV(
    model,
    params,
    max_iter=243,
    random_state=152,
)
X, y = make_classification()

In [8]:
%%time
search.fit(X, y)

CPU times: user 16.5 s, sys: 689 ms, total: 17.2 s
Wall time: 9min 58s


HyperbandSearchCV(aggressiveness=3,
         estimator=ReplayModel(batch_size=None,
      history=[{'model_id': 'bracket=1-0', 'params': {'train_split': None, 'optimizer__weight_decay': 0, 'optimizer__nesterov': True, 'optimizer__momentum': 0.16316316316316315, 'optimizer__lr': 1.666179250391986, 'optimizer': 'Adam', 'module__init': 'xavier_un...timizer_momentum=None, optimizer_nesterov=None,
      optimizer_weight_decay=None, train_split=None),
         max_iter=243,
         parameters={'module_init': ['xavier_uniform_', 'xavier_normal_', 'kaiming_uniform_', 'kaiming_normal_'], 'module_activation': ['ReLU', 'LeakyReLU', 'ELU', 'PReLU'], 'optimizer': ['SGD', 'SGD', 'SGD', 'SGD', 'SGD', 'Adam', 'Adam'], 'batch_size': [32, 64, 128, 256, 512], 'optimizer_lr': array([10.    ...rov': [True], 'optimizer_momentum': array([0.   , 0.001, ..., 0.999, 1.   ]), 'train_split': [None]},
         patience=False, random_state=152, scoring=None, test_size=None,
         tol=0.001)

In [29]:
def gcd(vals):
    for div in range(vals.min(), 0, -1):
        print(vals / div)
        if all(np.allclose(np.floor(x), x) for x in vals / div):
            return div
    return 0

max_iter = 243
values = np.array([max_iter // div for div in [3, 6, 9, 27]])
gcd(values)

[9.         4.44444444 3.         1.        ]
[10.125  5.     3.375  1.125]
[11.57142857  5.71428571  3.85714286  1.28571429]
[13.5         6.66666667  4.5         1.5       ]
[16.2  8.   5.4  1.8]
[20.25 10.    6.75  2.25]
[27.         13.33333333  9.          3.        ]
[40.5 20.  13.5  4.5]
[81. 40. 27.  9.]


1

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client


In [ ]:
today = "sim/2019-06-27-"

pre = today + f"{random_state}"
random_state = 152
with open(pre + "hyperband-history.json", 'w') as f:
    json.dump(search.history_, f)

Graphs:

* effect of patience
* cluster usage over time for various cluster sizes


In [ ]:
search = HyperbandSearchCV(
    model,
    params,
    max_iter=243,
    random_state=152,
    patience=True,
)

In [ ]:
%%time
search.fit(X, y)

In [ ]:
with open(pre + "hyperband-history.json", 'w') as f:
    json.dump(search.history_, f)